---

## 0. Setup and Loading package

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [2]:
import ipyleaflet
import geopandas as gpd
import rioxarray as rxr

from src import geograph
from src import geotimeline
from src.visualisation import geoviewer
from src.constants import UTM35N, DATA_DIR, ROIS

/home/users/svm/Code/gtc-biodiversity/env/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


---

## 1. Loading Data

In [3]:
# Parse geotif landcover data
chernobyl_path = lambda year: DATA_DIR / "chernobyl" / "esa_cci" / f"esa_cci_{year}_chernobyl.tif" 

# Parse ROIS
rois = gpd.read_file(ROIS)
cez = rois[rois["name"] == "Chernobyl Exclusion Zone"]

In [4]:
def clip_and_reproject(xrdata, clip_geometry=None, to_crs=UTM35N, x_res=300, y_res=300):
    
    if clip_geometry is not None:
        clipped_data = xrdata.rio.clip(clip_geometry)
    else:
        clipped_data = xrdata
        
    if to_crs is not None:
        reprojected_data = clipped_data.rio.reproject(to_crs, resolution=(x_res, y_res))
    else:
        reprojected_data = clipped_data
    
    return reprojected_data

In [5]:
# Loading raster data
years = list(range(2013,2015))
cez_rasters = {year: clip_and_reproject(rxr.open_rasterio(chernobyl_path(year)), clip_geometry=cez.geometry) 
              for year in years
             }

---

## 2. Creating `GeoGraph`

In [6]:
# Polygonising raster and transforming into graph
cez_graphs = {}
for year, raster in cez_rasters.items():
    print(f"Analysing year {year}")
    # Load geograph from the raster data (construction takes ~10s)
    cez_graphs[year] = geograph.GeoGraph(data=raster.data, 
                                transform=raster.rio.transform(), 
                                crs=UTM35N, 
                                mask = raster.data > 0,
                                connectivity=8)

Analysing year 2013


Step 1 of 2: Creating nodes and finding neighbours: 100%|██████████| 2003/2003 [00:12<00:00, 166.89it/s]
Step 2 of 2: Adding edges: 100%|██████████| 2003/2003 [00:00<00:00, 41990.63it/s]


Graph successfully loaded with 2003 nodes and 5140 edges.
Analysing year 2014


Step 1 of 2: Creating nodes and finding neighbours: 100%|██████████| 1999/1999 [00:11<00:00, 170.06it/s]
Step 2 of 2: Adding edges: 100%|██████████| 1999/1999 [00:00<00:00, 40553.78it/s]

Graph successfully loaded with 1999 nodes and 5117 edges.


---

## 3. Creating `Timeline` and identifying nodes:

In [7]:
cez_timeline = geotimeline.GeoGraphTimeline(cez_graphs)
# Perform node identification
cez_timeline.timestack()
# Classify node dynamics for the year 2014
cez_timeline.calculate_node_dynamics(2014);

---

## 4. Inspect in interactive viewer

In [8]:
viewer = geoviewer.GeoGraphViewer(small_screen=True)
viewer.add_layer(ipyleaflet.basemaps.Esri.WorldImagery)
viewer.add_graph(cez_timeline[2014], name='Polesia data', with_components=False)
viewer.enable_graph_controls()
viewer

GeoGraphViewer(center=[51.389167, 30.099444], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…